In [ ]:
!git clone https://github.com/SukritJaidee/pred_temp.git 
%cd /content/pred_temp
# !pip install -r requirements_colab_r1.txt

!pip install scikit-learn==1.2.0 #[1.0.1, 1.0.2, 0.23.2]
# !pip install APScheduler
# !pip install apscheduler

!pip install -q meteostat
!pip install -q mercantile
!pip install -q mpmath
# !pip install --pre pycaret
!pip install git+https://github.com/pycaret/pycaret.git#egg=pycaret

In [2]:
import pytz
import json
import warnings
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pycaret.regression import *
from tqdm.notebook import tqdm
from datetime import datetime, timedelta, date, tzinfo
warnings.filterwarnings("ignore")

from pred_temp.x3_ibm_api import x3_api
from pred_temp.tmd_api import tmd_weather
from pred_temp.x4_weather_api import x4_api
from pred_temp.loc_level10 import get_level10_pos
from pred_temp.x1_station_meteo_api import x1_api
from pred_temp.x2_point_meteo_api import x2_api, x1_station_x2

path_save = 'drive/MyDrive/result_weather/'
cols =  ['lat', 'lon', 'latitude', 'longitude', 'elevation', 'distance', 'temp', 'dwpt', 'rhum', 'wdir', 'wspd', 'year', 'month', 'day', 'hour', 'minute', 'temp.1', 'dwpt.1', 'rhum.1', 'wdir.1', 'wspd.1',
                    'pres', 'lat.1', 'lon.1', 'year.1', 'month.1', 'day.1', 'hour.1', 'minute.1', 'requestedLatitude', 'requestedLongitude', 'latitude.1', 'longitude.1', 'precip1Hour', 'precip6Hour', 'precip24Hour', 'pressureChange',
                    'pressureMeanSeaLevel', 'relativeHumidity', 'temperature', 'temperatureChange24Hour', 'temperatureMax24Hour', 'temperatureMin24Hour', 'temperatureDewPoint', 'temperatureFeelsLike', 'uvIndex',
                    'visibility', 'windDirection', 'windSpeed', 'lat.2', 'lon.2', 'lats', 'lons', 'temp_cs', 'wind_kphs', 'wind_degrees', 'wind_dirs', 'pressure_mbs', 'precip_mms', 'humiditys', 'clouds', 'feelslike_cs',
                    'windchill_cs', 'heatindex_cs', 'dewpoint_cs', 'will_it_rains', 'chance_of_rains', 'vis_kms', 'gust_kphs']

In [3]:
def cal_from_source(q1, now_datetime, format_date, saved_lr):
  #### x0
  diff_time, hour, duration = 0, 0, 24
  day = (datetime.now(pytz.timezone('Asia/Bangkok'))+timedelta(diff_time)).strftime("%Y-%m-%d")
  x0 = tmd_weather(q1[0], q1[1], day, hour, duration)
  # print('x0', end = ' '); print(x0.index[0], end = ' '); print(x0.index[-1]);
  #### x1
  start_date, end_date = -1, 1
  start = pd.to_datetime((now_datetime+timedelta(start_date)).strftime(format_date)) 
  end = pd.to_datetime((now_datetime+timedelta(end_date)).strftime(format_date))
  x1 = x1_api(q1, start, end)
  # print('x1', end = ' '); print(x1.index[0], end = ' '); print(x1.index[-1]);
  #### x2
  start_date, end_date = -1, 1
  start = pd.to_datetime((now_datetime+timedelta(start_date)).strftime(format_date)) 
  end = pd.to_datetime((now_datetime+timedelta(end_date)).strftime(format_date))
  x2 = x2_api(q1, start, end)
  if x2.shape[0] == 0:
    x2 = x1_station_x2(q1, start=-1, end=0)
    x2.drop(['latitude', 'longitude', 'elevation', 'distance'], axis=1, inplace=True)
    x2 = x2.reindex(columns=['temp', 'dwpt', 'rhum', 'wdir', 'wspd', 'pres', 'lat', 'lon', 'year', 'month', 'day', 'hour', 'minute'])
  # print('x2', end = ' '); print(x2.index[0], end = ' '); print(x2.index[-1]);
  #### x3
  apikey =  "2601b3bc3f684d2681b3bc3f687d26f8"
  format_date, start_date, end_date  = "%Y-%m-%d", -1, 1
  start = pd.to_datetime(now_datetime+timedelta(start_date)).strftime(format_date)
  end = pd.to_datetime(now_datetime+timedelta(end_date)).strftime(format_date)
  x3 = x3_api(q1, start, end, apikey)
  # print('x3', end = ' '); print(x3.index[0], end = ' '); print(x3.index[-1]);
  #### x4
  format_date, start_date, end_date = "%Y-%m-%d", 0, 1
  start = pd.to_datetime(now_datetime+timedelta(start_date)).strftime(format_date)
  end = pd.to_datetime(now_datetime+timedelta(end_date)).strftime(format_date)
  x4  = x4_api(q1, start, end)
  # print('x4', end = ' '); print(x4.index[0], end = ' '); print(x4.index[-1]);
  res = pd.concat([x0, x1, x2, x3, x4], axis=1,  join="inner")
  # print(res.shape); print('res', end = ' '); print(res.index[0], end = ' '); print(res.index[-1]);
  res1 = res.drop(['tc', 'rh', 'datatime'], axis=1)
  res1.columns = cols
  # print(res1.shape); print(display(res1.iloc[-1:,:]));
  try:
    unseen_pred = predict_model(saved_lr, data=res1)
  except:
    res1 = res1.astype('float32')
    unseen_pred = saved_lr.predict(res1)

  actual = res[['tc', 'datatime']]
  res2 = pd.concat([unseen_pred, actual], axis=1,  join="inner")
  data = res2.iloc[-1:,:]
  return data

In [ ]:
## Initial value
df = get_level10_pos() 
now_datetime  = datetime.now(pytz.timezone('Asia/Bangkok'))
format_date = "%Y-%m-%d %H:%M:%S"
models = 'Final Blend Model 20112022_950_ibm'
lats, lons, datetimes, pred_vals, tmd_vals = [], [], [],  [], []
saved_lr = load_model(models)
start_cp = datetime.now(pytz.timezone('Asia/Bangkok'))
for  i in tqdm(range(len(df))):
  try:
      lat, lon = df['lat_south'][i], df['lon_east'][i]
      q1 = (lat, lon)
      data = cal_from_source(q1, now_datetime, format_date, saved_lr)
      pred_val = data['prediction_label'][0]
      tmd = data['tc'][0]
      lats.append(lat); lons.append(lon);
      pred_vals.append(pred_val); tmd_vals.append(tmd);
  except:
      pred_val = data['temperature'][0]
      tmd = data['tc'][0]
      lats.append(lat); lons.append(lon);
      pred_vals.append(pred_val); tmd_vals.append(tmd);
      print(f"error @ lat = {round(q1[0],2)} lon = {round(q1[1],2)}")

success = True
end_cp = datetime.now(pytz.timezone('Asia/Bangkok'))
delta = end_cp - start_cp
sec = delta.total_seconds()
min = sec/60
print('minutes:', min)

format_save = "%d%m%Y-%H:%M"
now_datetime  = datetime.now(pytz.timezone('Asia/Bangkok'))
save_datetime = now_datetime.strftime(format_save) 
info_data = {'success':success, 'info':{'datetime':datetimes,'start_compute':start_cp,'end_compute':end_cp}, 'data':{'lat':lats,'lon':lons,'blend temp':pred_vals,'TMD temp':tmd_vals}}

df5 = pd.DataFrame({'lat':lats,'lon':lons,'blend temp':pred_vals,'TMD temp':tmd_vals})
df5.to_csv('/content/'+path_save+'data/onetime/'+save_datetime+'_data.csv' ,encoding='utf-8-sig')
info_data.to_csv('/content/'+path_save+'data/onetime/'+save_datetime+'_info.csv' ,encoding='utf-8-sig')

  0%|          | 0/25 [00:00<?, ?it/s]

(1100, 4)
   lon_west  lat_north  lat_south   lon_east
0  97.03125  20.632784  20.303418  97.382812
Transformation Pipeline and Model Successfully Loaded


  0%|          | 0/1100 [00:00<?, ?it/s]

error @ lat = 18.98 lon = 97.38
error @ lat = 18.65 lon = 97.38
error @ lat = 18.31 lon = 97.38
error @ lat = 11.18 lon = 97.38
error @ lat = 10.83 lon = 97.38
error @ lat = 10.49 lon = 97.38
error @ lat = 10.14 lon = 97.38
error @ lat = 9.8 lon = 97.38
error @ lat = 9.45 lon = 97.38
error @ lat = 9.1 lon = 97.38
error @ lat = 8.75 lon = 97.38
error @ lat = 8.41 lon = 97.38
error @ lat = 8.06 lon = 97.38
error @ lat = 7.71 lon = 97.38
error @ lat = 7.36 lon = 97.38
error @ lat = 7.01 lon = 97.38
error @ lat = 6.66 lon = 97.38
error @ lat = 6.32 lon = 97.38
error @ lat = 5.97 lon = 97.38
error @ lat = 5.62 lon = 97.38
error @ lat = 19.64 lon = 97.73
error @ lat = 19.31 lon = 97.73
error @ lat = 18.98 lon = 97.73
error @ lat = 18.65 lon = 97.73
error @ lat = 18.31 lon = 97.73
error @ lat = 11.18 lon = 97.73
error @ lat = 10.83 lon = 97.73
error @ lat = 10.49 lon = 97.73
error @ lat = 10.14 lon = 97.73
error @ lat = 9.8 lon = 97.73
error @ lat = 9.45 lon = 97.73
error @ lat = 9.1 lon = 97